In [3]:
from nba_api.stats.static import players,teams
from nba_api.stats.endpoints import PlayerVsPlayer, TeamVsPlayer, leaguegamefinder, playbyplay
import pandas as pd 
import numpy as np
import json
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

In [5]:
steph_id = players.find_players_by_full_name('Steph')[1]['id']
pat_id = players.find_players_by_full_name('Patrick')[0]['id']


In [3]:
PlayerVsTeam = TeamVsPlayer(team_id='1610612740', vs_player_id=steph_id).get_normalized_dict()

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

# how to access parameters of api calls

In [42]:
params = TeamVsPlayer(team_id='1610612740', vs_player_id=steph_id).parameters

# An example of TeamVsPlayer to DataFrame

In [44]:
normalized_dict = TeamVsPlayer(team_id='1610612740', vs_player_id=steph_id, season='2015-16').get_normalized_dict()
df = pd.DataFrame.from_dict(normalized_dict, orient = 'index')
TeamVsPlayer = df.transpose()
TeamVsPlayer['Overall']

TypeError: 'DataFrame' object is not callable

# An example of PlayerVsPlayer to DataFrame

In [28]:
data = PlayerVsPlayer(player_id = steph_id, vs_player_id = pat_id).get_normalized_dict()
df = pd.DataFrame.from_dict(data, orient = 'index')
PlayerVsPlayer = df.transpose()

In [13]:
pd.DataFrame.from_dict(df['ShotAreaOnCourt'][5],orient = 'index').transpose()

,GROUP_SET,PLAYER_ID,PLAYER_NAME,VS_PLAYER_ID,VS_PLAYER_NAME,COURT_STATUS,GROUP_VALUE,FGM,FGA,FG_PCT,CFID,CFPARAMS
0,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Above the Break 3,0,7,0,100,201976


In [14]:
ShotAreaOnCourt_DF = pd.DataFrame(pd.DataFrame.from_dict(df['ShotAreaOnCourt'][0],orient = 'index').transpose())

In [16]:
# for i in df['ShotAreaOnCourt']:
#     ShotAreaOnCourt_DF.append(pd.DataFrame.from_dict(df['ShotAreaOnCourt'][i],orient = 'index').transpose())
    

In [17]:
pd.DataFrame.from_dict(df['ShotAreaOnCourt'][6],orient = 'index').transpose()

,GROUP_SET,PLAYER_ID,PLAYER_NAME,VS_PLAYER_ID,VS_PLAYER_NAME,COURT_STATUS,GROUP_VALUE,FGM,FGA,FG_PCT,CFID,CFPARAMS
0,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Backcourt,0,0,0,101,201976


In [18]:
pd.concat([pd.DataFrame.from_dict(df['ShotAreaOnCourt'][i],orient = 'index').transpose() for i in range(7)],ignore_index=True)







###########

,GROUP_SET,PLAYER_ID,PLAYER_NAME,VS_PLAYER_ID,VS_PLAYER_NAME,COURT_STATUS,GROUP_VALUE,FGM,FGA,FG_PCT,CFID,CFPARAMS
0,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Restricted Area,1,1,1,95,201976
1,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,In The Paint (Non-RA),1,2,0.5,96,201976
2,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Mid-Range,1,2,0.5,97,201976
3,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Left Corner 3,1,1,1,98,201976
4,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Right Corner 3,0,0,0,99,201976
5,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Above the Break 3,0,7,0,100,201976
6,Vs. Player,201939,Stephen Curry,201976,"Beverley, Patrick",On,Backcourt,0,0,0,101,201976


In [19]:
len(df['ShotAreaOnCourt'])

9

In [20]:
df.columns

Index(['Overall', 'OnOffCourt', 'ShotDistanceOverall', 'ShotDistanceOnCourt',
       'ShotDistanceOffCourt', 'ShotAreaOverall', 'ShotAreaOnCourt',
       'ShotAreaOffCourt', 'PlayerInfo', 'VsPlayerInfo'],
      dtype='object')

In [21]:
df['ShotAreaOverall'][0]

{'GROUP_SET': 'Shot Area',
 'GROUP_VALUE': 'Restricted Area',
 'PLAYER_ID': 201939,
 'PLAYER_NAME': 'Stephen Curry',
 'FGM': 11,
 'FGA': 16,
 'FG_PCT': 0.688,
 'CFID': 102,
 'CFPARAMS': None}

In [22]:
pd.DataFrame.from_dict(df['Overall'][1],orient = 'index').transpose()

,GROUP_SET,GROUP_VALUE,PLAYER_ID,PLAYER_NAME,GP,W,L,W_PCT,MIN,FGM,...,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,CFID,CFPARAMS
0,Overall,Patrick Beverley,201976,Patrick Beverley,7,5,2,0.714,211.395,18,...,12,2,1,23,21,43,38,156.4,85,Patrick Beverley


In [23]:
df['PlayerInfo']

0    {'PERSON_ID': 201939, 'FIRST_NAME': 'Stephen',...
1                                                 None
2                                                 None
3                                                 None
4                                                 None
5                                                 None
6                                                 None
7                                                 None
8                                                 None
Name: PlayerInfo, dtype: object

In [71]:
nba_teams = teams.get_teams()

# Select the dictionary for the Pacers, which contains their team ID
golden = [team for team in nba_teams if team['abbreviation'] == 'GSW'][0]
golden_id = golden['id']
print(f'pacers_id: {golden_id}')

pacers_id: 1610612744


In [72]:
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=golden_id,
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular)  

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

In [73]:
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]


In [75]:
df


,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0021900096,2,12,0,1,10:42 PM,12:00,None,None,None,None,None
1,0021900096,4,10,0,1,10:42 PM,12:00,Jump Ball Cauley-Stein vs. Whiteside: Tip to T...,None,None,None,None
2,0021900096,7,6,2,1,10:42 PM,11:42,Cauley-Stein S.FOUL (P1.T1) (K.Mauer),None,None,None,None
3,0021900096,9,3,11,1,10:43 PM,11:42,None,None,MISS Whiteside Free Throw 1 of 2,None,None
4,0021900096,10,4,1,1,10:43 PM,11:42,None,None,Trail Blazers Rebound,None,None
5,0021900096,11,3,12,1,10:43 PM,11:42,None,None,Whiteside Free Throw 2 of 2 (1 PTS),1 - 0,-1
6,0021900096,12,1,79,1,10:43 PM,11:27,Paschall 27' 3PT Pullup Jump Shot (3 PTS) (Poo...,None,None,1 - 3,2
7,0021900096,14,2,1,1,10:43 PM,11:08,None,None,MISS Lillard 26' 3PT Jump Shot,None,None
8,0021900096,15,4,0,1,10:43 PM,11:06,None,None,Whiteside REBOUND (Off:1 Def:0),None,None
9,0021900096,16,1,5,1,10:43 PM,11:05,None,None,Whiteside 4' Layup (3 PTS),3 - 3,TIE


In [1]:
headers = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}

In [7]:



trial = TeamVsPlayer(vs_player_id=steph_id,team_id='1610612737',season='2016-17',headers=headers).get_normalized_dict()

df = pd.DataFrame.from_dict(trial, orient = 'index').transpose()

In [72]:
df.iloc[2]['OnOffCourt']

,Overall,vsPlayerOverall,OnOffCourt,ShotDistanceOverall,ShotDistanceOnCourt,ShotDistanceOffCourt,ShotAreaOverall,ShotAreaOnCourt,ShotAreaOffCourt
0,"{'GROUP_SET': 'Overall', 'GROUP_VALUE': 'Atlan...","{'GROUP_SET': 'Overall', 'GROUP_VALUE': 'Steph...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Res...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
1,None,None,"{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'In ...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
2,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Mid...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
3,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Lef...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
4,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Rig...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
5,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Abo...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
6,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Bac...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
7,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...",None,"{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...",None,None,None
8,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...",None,None,None,None,None


In [76]:
nba_teams_info = pd.concat([pd.DataFrame.from_dict(teams.get_teams()[i], orient='index').transpose() for i in range(30)],ignore_index=True)


In [86]:
StephVsTeam_df = pd.DataFrame(columns = nba_teams_info['full_name'], index=['2018-19','2017-18','2016-17','2015-14','2014-13','2013-12','2012-11','2011-12','2010-11','2009-10'])


In [87]:
StephVsTeam_df

full_name,Atlanta Hawks,Boston Celtics,Cleveland Cavaliers,New Orleans Pelicans,Chicago Bulls,Dallas Mavericks,Denver Nuggets,Golden State Warriors,Houston Rockets,Los Angeles Clippers,...,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Oklahoma City Thunder,Toronto Raptors,Utah Jazz,Memphis Grizzlies,Washington Wizards,Detroit Pistons,Charlotte Hornets
2018-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
StephVsTeam_df.loc['2018-2019']['Atlanta Hawks']

nan

In [82]:
df

,Overall,vsPlayerOverall,OnOffCourt,ShotDistanceOverall,ShotDistanceOnCourt,ShotDistanceOffCourt,ShotAreaOverall,ShotAreaOnCourt,ShotAreaOffCourt
0,"{'GROUP_SET': 'Overall', 'GROUP_VALUE': 'Atlan...","{'GROUP_SET': 'Overall', 'GROUP_VALUE': 'Steph...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Res...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
1,None,None,"{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'In ...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
2,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Mid...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
3,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Lef...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
4,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Rig...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
5,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Abo...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
6,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Bac...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...","{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612..."
7,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...",None,"{'GROUP_SET': 'Vs. Player', 'TEAM_ID': 1610612...",None,None,None
8,None,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...",None,None,None,None,None


In [85]:
StephVsTeam_df.head()


full_name,Atlanta Hawks,Boston Celtics,Cleveland Cavaliers,New Orleans Pelicans,Chicago Bulls,Dallas Mavericks,Denver Nuggets,Golden State Warriors,Houston Rockets,Los Angeles Clippers,...,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Oklahoma City Thunder,Toronto Raptors,Utah Jazz,Memphis Grizzlies,Washington Wizards,Detroit Pistons,Charlotte Hornets
2018-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
nba_teams_info = pd.concat([pd.DataFrame.from_dict(teams.get_teams()[i], orient='index').transpose() for i in range(30)],ignore_index=True)

In [ ]:
def filter_col(df):
    df.drop(['Overall','vsPlayerOverall','ShotDistanceOverall','ShotDistanceOffCourt','ShotAreaOverall','ShotAreaOffCourt'])
    return df


def insert_PlayerVsTeam(StephVsTeam_df,player_id,team_info):
    seasons = ['2018-19','2017-18','2016-17','2015-14','2014-13','2013-12','2012-11','2011-12','2010-11','2009-10']
    for i in seasons:
        team_id = team_info['id'][0]
        team_name = team_info['full_name'][0]
        dict_team = TeamVsPlayer(vs_player_id=player_id,team_id=team_id,season=i,headers=headers).get_normalized_dict()
        dict_to_df = pd.DataFrame.from_dict(dict_team, orient = 'index').transpose()
        filtered_col = filter_col(dict_to_df)
        StephVsTeam_df.loc[i][team_name] = filtered_col
    

In [134]:
nba_players_info = pd.concat([pd.DataFrame.from_dict(players.get_players()[i], orient='index').transpose() for i in range(len(players.get_players()))],ignore_index=True)

In [ ]:
nba_playersnba_players_info['is_active'] == True

In [135]:
def is_active(players_info):
    

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
5,949,Shareef Abdur-Rahim,Shareef,Abdur-Rahim,False
6,76005,Tom Abernethy,Tom,Abernethy,False
7,76006,Forest Able,Forest,Able,False
8,76007,John Abramovic,John,Abramovic,False
9,203518,Alex Abrines,Alex,Abrines,True


In [99]:
def insert_PlayerVsPlayer(StephVsPlayers_df,player_id,vs_player_id):
    

'Atlanta Hawks'

In [121]:
data = PlayerVsPlayer(player_id = steph_id, vs_player_id = pat_id,headers=headers,season='2015-16').get_normalized_dict()
df = pd.DataFrame.from_dict(data, orient = 'index').transpose()

In [133]:
df.iloc[5]['ShotAreaOnCourt']

{'GROUP_SET': 'Vs. Player',
 'PLAYER_ID': 201939,
 'PLAYER_NAME': 'Stephen Curry',
 'VS_PLAYER_ID': 201976,
 'VS_PLAYER_NAME': 'Beverley, Patrick',
 'COURT_STATUS': 'On',
 'GROUP_VALUE': 'Above the Break 3',
 'FGM': 2,
 'FGA': 9,
 'FG_PCT': 0.222,
 'CFID': 100,
 'CFPARAMS': '201976'}

In [108]:
df

,Overall,OnOffCourt,ShotDistanceOverall,ShotDistanceOnCourt,ShotDistanceOffCourt,ShotAreaOverall,ShotAreaOnCourt,ShotAreaOffCourt,PlayerInfo,VsPlayerInfo
0,"{'GROUP_SET': 'Overall', 'GROUP_VALUE': 'Steph...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Res...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'PERSON_ID': 201939, 'FIRST_NAME': 'Stephen',...","{'PERSON_ID': 201976, 'FIRST_NAME': 'Patrick',..."
1,"{'GROUP_SET': 'Overall', 'GROUP_VALUE': 'Patri...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'In ...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
2,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Mid...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
3,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Lef...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
4,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Rig...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
5,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Abo...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
6,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Shot Area', 'GROUP_VALUE': 'Bac...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None
7,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...","{'GROUP_SET': 'Vs. Player', 'PLAYER_ID': 20193...",None,None,None,None,None,None
8,None,None,"{'GROUP_SET': 'Shot Distance (5ft)', 'GROUP_VA...",None,None,None,None,None,None,None
